support from: https://github.com/bradleypallen/keras-quora-question-pairs

In [6]:
%matplotlib inline

import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML

import nltk            # natural language tool kit
import numpy as np     # support for large data structures
import pandas as pd    # data structure support
import string          # various string functions
import difflib         # classes and functions for comparing sequences
import utils           # word processing functions and distance functions, pretty printing and data loading


from importlib import reload       # reload external files

import tensorflow as tf
assert(tf.__version__.startswith("1."))

# rnn code
import rnnlstm

# Helper libraries
from shared_lib import vocabulary, tf_embed_viz 

# set a default voab size
V=10000


import plotly.plotly
import plotly.graph_objs as go

# need to look into plotly issues with this line in python3
#plotly.offline.init_notebook_mode() # run at the start of every ipython notebook

from sklearn.metrics import log_loss    # used in measurement / scoring
from sklearn.metrics import classification_report   # among other things, provides accuracy and f1

In [7]:
# Generate training data dataframe
# train_lite for testing approach
train = pd.read_csv('Data/train_lite.csv') #index_col=0
# full train for when working
# train = pd.read_csv('Data/train.csv') #index_col=0

print (len(train))
train.head()

10000


,Unnamed: 0,Unnamed: 0.1,id,qid1,qid2,question1,question2,is_duplicate
0,0,57674,57674,5725,38477,How can I get a list of my Gmail accounts?,How do I find my list of GMail addresses?,1
1,1,185568,185568,283254,283255,What are the symptoms of child abuse?,What are signs of child abuse?,1
2,2,359111,359111,400936,488715,What are some examples of selfishness?,Do we all occasionally tend to emotional masoc...,0
3,3,204216,204216,281934,307050,Is Qnet a scam?,Where is the registered office of Qnet in Mumbai?,0
4,4,5649,5649,11103,11104,What do Americans think about Donald Trump?,What do you think about Donald Trump pick?,1


#### Prepping Questions

In [58]:
reload(vocabulary)
reload(utils)
question1 = train.question1.tolist()
question2 = train.question2.tolist()
is_duplicate = train.is_duplicate.tolist()

# print('Question pairs: %d' % len(question1))

# build word index
questions = question1 + question2

vocab = utils.build_vocab(questions, V)

q1_sequences = utils.preprocess_sentences(question1, vocab)
q2_sequences = utils.preprocess_sentences(question2, vocab)

word_index = vocab.ordered_words()

print("Words in index: %d" % len(word_index))



Words in index: 10000


#### lets start building tensor flow
build graph

Within BuildCoreGraph:

Build embedding layer; question 1 and question 2 feed into relu layer

Construct RNN/LSTM cell and recurrent layer (leverage a3 here as is (at least for now))

Build output

Finalize loss computation

In [63]:
reload (rnnlstm)

# building graph based on a3 assigment
TF_GRAPHDIR = "tf_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

# V, H, and num_layers are all things we can play with...
# default corpus is 'brown'
# for now, want to see running with this data
# set hyperparameters
lm = rnnlstm.RNNLM(V=V, H=200, num_layers=2)

lm.BuildCoreGraph()


AttributeError: 'tuple' object has no attribute 'get_shape'

In [ ]:
# NOT YET REVIEWED

lm.BuildTrainGraph()
lm.BuildSamplerGraph()
summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)